In [1]:
import numpy as np
import pandas as pd

PATH_COLLECTION_DATA = 'data/subtask_4b/subtask4b_collection_data.pkl'

df_collection = pd.read_pickle(PATH_COLLECTION_DATA)
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [2]:
PATH_QUERY_DATA = 'data/dev-tweets/subtask4b_query_tweets_dev.tsv'
df_query = pd.read_csv(PATH_QUERY_DATA, sep = '\t')
df_query.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [3]:
sub_df_query = df_query[:100]
sub_df_query.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [4]:
import os
base_name = "models/bi-encoder"
dir_list = os.listdir(base_name)
dir_list.sort()

latest_checkpoint = dir_list[-1]

model_name = f"{base_name}/{latest_checkpoint}"

print(model_name)

models/bi-encoder/checkpoint-7712


In [5]:
from tqdm import tqdm
from rankers.bi_encoder_ranker import BiEncoderRanker

tqdm.pandas()

cord_uids = df_collection[:]['cord_uid'].tolist()
corpus = df_collection[:]['abstract'].tolist()
bi_enc_ranker = BiEncoderRanker(model_name, corpus)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: models/bi-encoder/checkpoint-7712


Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:

def get_top_cord_uids(query):
    doc_scores = bi_enc_ranker.get_scores(query)
    indices = np.argsort(-doc_scores[0])[:1000]
    bi_enc_topk = [cord_uids[x] for x in indices]
    return bi_enc_topk

# Retrieve topk candidates using the BM25 model
sub_df_query['cross'] = sub_df_query['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))

 14%|█▍        | 14/100 [00:00<00:01, 68.18it/s]

['hg3xpej0', 'styavbvi', 'nksd3wuw', '5hxsagx6', '6mfd3n4s', 'jrqlhjsm', '00ia8k0b', 'bqn29m9k', 'ny7wkswf', '8t2tic9n', 'is9a77dk', 'wfa5fb3r', 'krtqq8uo', 'ktv6fk7q', 'yrq57wgs', 'uxz185vr', '3qvh482o', '0rrhgz26', 'rthsl7a9', 'oz047qmf', '3i82sxk8', 'ngb7pbyl', '8cd5z4di', 'x7qlnugx', 'mamtxi9v', '00ugdhvf', '0ml57rhr', 'dcfqvp4p', 'b1g23kz1', 'rnq3k34m', '82y56t7d', '3cl3q80m', 'i8jv5iqo', '7015mpuh', '0041cay9', 'oe7fzahb', 'omz42d9y', 'dogsza0f', 'tdwkxwod', 'sg1e404x', 'n2kn7o67', 'es8l29ub', 'q5veu992', 'wqicq5ry', 'm20lrom9', 'vymre7uv', 'gtbegs4r', 'jaamms1g', 'e7gq365q', '7oipjzq1', 'efbj3uur', 'cn0hqvsz', 'pjllicm7', '5tpkcd5z', 'lcqxnrge', '59up4v56', 'r9qtzdvk', '8g2mr8o1', 'vge26qe9', 'ivue7zob', '86xwnpde', 'o6i5ca7h', 'ix0xi5jb', 'bn22k0p3', '22q7tngp', 'x3q2dgdu', 'c7ci5w56', 'z1onkz6u', '811lov8f', 'ynaxwnlp', '5kchpjx0', '0phcscz8', 'mncip6jj', 'niqka1hh', 'jwei2g97', 'rtus1qyw', 'eyhgtpe0', 'oyplh9cn', 'kt615lm4', 'llo64qbp', 'hduwjbtq', 'tgrh7ap4', 'edr7hrhp', 'cm

 29%|██▉       | 29/100 [00:00<00:01, 68.49it/s]

['s3o9v1fl', 'm4vu77v6', 'ksgdjdh1', '509h7pnh', 'vvuaegse', 'fm8koqjd', '7k8nlea3', 'gvihovvq', '5525dp31', 'ktymj1no', 'wl4n02rf', 'mru0mcup', 'i9nchww2', 'gorp9n1g', 'ozj6zju8', 'kc6dls2z', 'wvfw94n1', '70m9l3y0', 'zmk8bbcd', '9nc90z3f', 'eakfj0wv', 'cvxni7cd', '3l3zs90e', 'b03s7wsw', 'pksg0exj', 'yj652emy', '4kfl29ul', '84xjcyvu', 'iab4qya4', 'whkfvd7l', 'ah8mzt1g', '1k0qpuys', 'vrttjv2b', 'tr2139vg', '99irhqbu', 'h29zzmla', 'd0nacth6', '9megonpn', 'gdyntyxk', '493uvn41', '3jyokb6c', 'p10cya94', '4b6jrycu', 'qp34apct', '53ts46vr', '3o7rd8pt', 'vrrlub35', 'ckswlo9o', 'qa4njmyt', 'dwcu6vyp', 'bbhb1cre', '1lzimff2', 'ijdk1nnc', 'tdxrx9se', '4seogrqq', 'a9lw6vrs', 'cmnere6u', 'a7blklzz', '46fbb81c', 'u0lvjlbs', '9lc20ohq', 'jvqlmqsy', 'utgnwox8', 'opw9cmbe', '29z4q4fs', 'efkl93ta', '5hhbdzam', '2uu19itv', '9oxajwkk', 'dfhxcqr8', 'jc5ik84m', 'ekgqdjlk', 'r1wh54q1', '62esadjt', 'jisznsr3', '10l12wgu', 'vkbsxz7r', 'wd13eb5s', 't9hbbqay', 'iz74o9de', '7objlrqa', '7a7oi5dt', '9wn91fj2', 'ca

 44%|████▍     | 44/100 [00:00<00:00, 68.30it/s]

...

 89%|████████▉ | 89/100 [00:01<00:00, 65.11it/s]

['w4dyadpa', '3014grzd', 'rl5g8iqm', 'wcyqks52', 'f3w4v5ej', 'ubjd0052', 'k4k3kcrh', 'okbjepet', 'r0pzpont', '0rftk787', '8sed3htx', 'yfuddx55', 'gj8p22hq', '32ua8wb6', '8hvgsico', '4ifng0a6', 'jd3np9n4', 'ccj1dt86', 'hlv5zb0s', 'xt696aq9', '6vk9mvv7', 'cmc02ybp', 'tx7ql71r', 'jbpocl1k', '1oi0k3x6', 'a47sz3r5', 'hs7l8ih1', 'ldbscq4s', 'lsgm7y5t', 'lsvf3hl6', 'dew7ug4s', 'hyx95otb', 'dcpnwc2b', 'm8hkj1dm', 'vf7xj5qt', '0ya8qvka', 'vsinwqnr', 'zgbzb8bl', 'thaje9nj', '208w0eyz', 'liip39ku', 'mzn4jztt', '0hmwg7io', 'pkimbhz9', '2bm389su', 'x0me00m0', 'ft4f28qp', 'h400e0kk', 'cb3u1s7s', 'f2det6w2', '9z6k1hx8', '9i1u93ht', 'd4p5r746', '2hfkrl8m', 'n5sei1oc', 'agvrw27o', '85k68r5h', 'esizmxoz', '6gcgzx36', 'n0p8osku', 'e8fhlo37', 'uw4jayha', 'cqjkjkt3', 'n040i865', 'cw3epg39', 'qgt7tau1', 'fhc1qf8d', '2aowm09g', '3vx85o8n', 'b3doorzq', 'ml8j19l7', 'fen3nmr2', 'v85v5o51', 'jcueu1xh', 'b1vlaly1', '7iqyaitx', 'rbau5dc2', 't1rtrbtd', 'dogie34u', 'n0uy6hd2', '5w2yxjvx', 'flgmnr70', '1itor6ic', 'rv

100%|██████████| 100/100 [00:01<00:00, 65.28it/s]

['ovgk0wsn', 'ywvk6gu6', 'qhbuvt1l', 'v6w9mlsa', 'o1mg23wb', 'nq79qw82', '9wvlt53w', '8sb8brew', 'oc584x1k', '5t4twnpm', '0leez9la', 'cwv6u0y9', '9bvetodx', 'v5vwypya', 'fatm351r', '6mdimxnk', 'p0w1nyof', 'nbn44sxq', 'o45sziw1', 'ipon95io', '69afkw7x', '918wd3ez', 'k1adcls8', '05lgp662', 'kkbed30m', 'f9jr9kki', '0jl6qu0i', 'ey34e59f', 's3qkscw7', '8n4zf9oo', 'fgj2lvbg', 'elj952zc', 'ho5n7n90', '0e97tdez', 'ihq8gdwj', 'hb2bpyag', '3tlv2b5x', 'tmgmqtjq', 'ycklcks9', 'epzpua9t', 'eebsywlz', '7vadpp3d', '9yl6lg4y', 'mzpzz8sr', 'on6ss822', 'phl4mmpt', 'sxq2063i', 'jeufhpkq', 'vdd8uuv8', 'r7bmtloy', '02u91ess', 'yyfsnthj', 'wm3l1849', 'vgvcd02j', 'qy4gsvac', '7o97t2t7', 'i6o608fm', 'e3e48idb', 'x14iywtr', '4fkg2pfm', 'kjd2t69f', 'x5htdsbn', '9getsuci', 'a242g5kg', 'vwvzxlqh', 'i5j4nmjz', 'xmiefdez', 'mzqo4wjo', 't9e774ed', '6zfpcm4j', 'dzkhc9q5', 'peiqusin', 'zzyibdy7', '83re9176', '0zphhxxh', '9mvk86q6', 'yku0iduy', '7e3kiox2', '2dvljm27', 'm1sf159a', '96lxmeqx', 'nrl6jkpc', 'sfr2nsmi', 'nx


/tmp/ipykernel_13673/509693874.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_query['cross'] = sub_df_query['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))


In [7]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

results = get_performance_mrr(sub_df_query, 'cord_uid', 'cross')
# Printed MRR@k results in the following format: {k: MRR@k}
print(">>>")
print(results)
print("<<<")

>>>
{1: np.float64(0.47), 5: np.float64(0.5435), 10: np.float64(0.5513849206349206)}
<<<


/tmp/ipykernel_13673/1153424369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
